In [22]:
!pip3 install langchain_huggingface
!pip3 install yfiles_jupyter_graphs
!pip3 install neo4j
!pip3 install langchain_experimental
!pip3 install langchain_core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from typing import List
from langchain_community.graphs import Neo4jGraph
from langchain.docstore.document import Document
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import RetrievalQA
import gradio as gr
import os



In [24]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Password1!"

graph = Neo4jGraph()


In [25]:
text = """Emily is an employee at TechNova, a leading technology company based in Silicon Heights. She has been working there for the past four years as a software developer. James is also an employee at TechNova, where he works as a data analyst. He joined the company three years ago after completing his undergraduate studies. TechNova is a renowned technology company that specializes in developing innovative software solutions and advanced artificial intelligence systems. The company boasts a diverse team of talented professionals from various fields. Both Emily and James are highly skilled experts who contribute significantly to TechNova's achievements. They collaborate closely with their respective teams to create cutting-edge products and services that cater to the dynamic needs of the company's clients."""

# Initialize TokenTextSplitter with desired parameters
text_splitter = TokenTextSplitter(
    chunk_size=200,  # Adjust chunk size as needed
    chunk_overlap=20  # Adjust overlap to maintain context
)

# Convert the formatted text into a list of Document objects
documents = [Document(page_content=text)]

# Split the text into chunks
texts = text_splitter.split_documents(documents)

# Print the split chunks
for i, chunk in enumerate(texts):
    print(f"Chunk {i+1}:\n{chunk}\n")



Chunk 1:
page_content='Emily is an employee at TechNova, a leading technology company based in Silicon Heights. She has been working there for the past four years as a software developer. James is also an employee at TechNova, where he works as a data analyst. He joined the company three years ago after completing his undergraduate studies. TechNova is a renowned technology company that specializes in developing innovative software solutions and advanced artificial intelligence systems. The company boasts a diverse team of talented professionals from various fields. Both Emily and James are highly skilled experts who contribute significantly to TechNova's achievements. They collaborate closely with their respective teams to create cutting-edge products and services that cater to the dynamic needs of the company's clients.'



In [26]:

llm = Ollama(model="llama3")

llm_transformer = LLMGraphTransformer(
    llm=llm, 
    allowed_nodes=["Author", "Book", "Publisher"],
    allowed_relationships=["WRITTEN_BY", "PUBLISHED_BY"],
)



In [27]:
# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(texts)
print("graph documents", graph_documents)


graph documents [GraphDocument(nodes=[Node(id='software developer', type='Book'), Node(id='Emily', type='Author'), Node(id='TechNova', type='Publisher'), Node(id='data analyst', type='Book'), Node(id='James', type='Author')], relationships=[Relationship(source=Node(id='Emily', type='Author'), target=Node(id='software developer', type='Book'), type='WRITTEN_BY'), Relationship(source=Node(id='James', type='Author'), target=Node(id='data analyst', type='Book'), type='WRITTEN_BY')], source=Document(page_content="Emily is an employee at TechNova, a leading technology company based in Silicon Heights. She has been working there for the past four years as a software developer. James is also an employee at TechNova, where he works as a data analyst. He joined the company three years ago after completing his undergraduate studies. TechNova is a renowned technology company that specializes in developing innovative software solutions and advanced artificial intelligence systems. The company boast

In [28]:
print(f"Nodes:{graph_documents[0].nodes}") # this is to see Nodes of generated graph
print("-----------------------------------------------------------------")
print(f"Relationships:{graph_documents[0].relationships}") 


Nodes:[Node(id='software developer', type='Book'), Node(id='Emily', type='Author'), Node(id='TechNova', type='Publisher'), Node(id='data analyst', type='Book'), Node(id='James', type='Author')]
-----------------------------------------------------------------
Relationships:[Relationship(source=Node(id='Emily', type='Author'), target=Node(id='software developer', type='Book'), type='WRITTEN_BY'), Relationship(source=Node(id='James', type='Author'), target=Node(id='data analyst', type='Book'), type='WRITTEN_BY')]


In [29]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()


GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
graph.add_graph_documents(graph_documents)
print("Documents successfully added to Graph DataBase")

Documents successfully added to Graph DataBase


In [33]:
# # Ensure valid labels for graph documents
# for doc in graph_documents:
#     print(doc.nodes)
#     if not doc.nodes:
#         doc.nodes = ['Document']  # Assign a default label if none exists


# Store to neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)
print("Documents successfully added to Graph DataBase")


Documents successfully added to Graph DataBase


In [34]:
#load the embedding model

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name  = "BAAI/bge-base-en-v1.5")

In [35]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [36]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)


In [37]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [38]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=vector_index.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)



In [39]:
# Define the function for querying patient details
def query_patient_details(query):
    if query is None:
        return "Error: Query cannot be None"
    try:
        result = qa_chain({"query": query})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

In [40]:
# Create a Gradio interface
interface = gr.Interface(
    fn=query_patient_details,        # Function to call
    inputs=gr.Textbox(label="Enter your question"),  # Input textbox
    outputs=gr.Textbox(label="Answer")   # Output textbox
)

# Launch the interface
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
